In [ ]:
import os
import glob
from openai import OpenAI
from dotenv import load_dotenv
import gradio as gr

In [ ]:
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')

openai = OpenAI()

In [ ]:

employees = glob.glob('knowledge-base/employees/*')
products = glob.glob('knowledge-base/products/*')

print(products)

In [ ]:
context = {}

for employee_name in employees:
    emp_name = employee_name.split(" ")[-1][:-3]
    doc = ""
    with open(employee_name, 'r') as f:
        doc = f.read()
    context[emp_name] = doc 


In [ ]:
for product_name in products:
    prod_name = product_name.split(os.sep)[-1][:-3]
    doc = ""
    with open(product_name, 'r') as f:
        doc = f.read()
    context[prod_name] = doc


In [ ]:
def get_relevant_context(user_prompt):
    relevant = []

    for relevant_name, relevant_description in context.items():
        if relevant_name.lower() in user_prompt.lower():
            relevant.append(relevant_description)
    
    return relevant

In [ ]:
def add_more_context(user_prompt):
    relevant_context = get_relevant_context(user_prompt)

    if relevant_context:
        user_prompt += "\n\nThe following additional context might be relevant in answering this question:\n\n"

        for relevant in relevant_context:
            user_prompt += relevant + "\n\n"

    return user_prompt 



In [ ]:
system_message = "You are an expert in answering accurate questions about Insurellm, the Insurance Tech company. Give brief, accurate answers. If you don't know the answer, say so. Do not make anything up if you haven't been provided with relevant context."

In [ ]:
def get_llm_response(user_message, history):

    conversation = [{'role': 'system', 'content': system_message}] + history 
    user_prompt = add_more_context(user_message)
    conversation.append({'role': 'user', 'content': user_prompt})

    response = openai.chat.completions.create(
        model= 'gpt-4o-mini',
        messages= conversation,
        stream= True
    )
    output = ""
    for stream in response:
        output += stream.choices[0].delta.content or ""
    yield output

In [ ]:
view = gr.ChatInterface(get_llm_response, type="messages").launch()